# Graphanalyse Co-Zitationen

Zunächst muss die folgende Zelle zum Laden der Module ausgeführt werden. Danach müssen die weiteren Zellen nach und nach ausgeführt werden.

In [13]:
import os
import pandas as pd
from collections import Counter, OrderedDict
import networkx as nx
import nx_altair as nxa
import altair as alt
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics.pairwise import cosine_similarity
import spacy 

low_memory=False

# Daten einlesen
Der fogende Code liest den Export der PatentsView-Datenbank aus und speichert diese in einem Pandas-Dataframe.

In [14]:
# load data
filepath = os.path.join(os.getcwd(), 'patents_with_title.csv')
data = pd.read_csv(filepath , sep = ";", dtype={'patent_number': str, 'cited_patent_number': str})


D:\Programme\Anaconda\envs\mining\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7,8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Datenbereinigung
In dem Export befinden sich Duplikate, welche aus der Normalisierung der Daten erstanden sind. Diese Duplikate sind z. B. auf die Normalisierung der Erfinder zurückzuführen.
Diese Duplikate werden im Folgenden entfernt. Zusätzlich werden fehlende Werte z. B. in der Spalte der Anmelder durch "0" ersetzt. Die Fehlenden Werte kommen bei den Anmelder z. B. zustande, wenn es sich um Erfinder handelt, die die Erfindung nicht zusammen mit Unternehmen anmelden.

In [15]:
data = data.drop_duplicates(subset=["patent_number","cited_patent_number"])
data = data[data['cited_patent_number'].notna()]
count = 0
data.fillna("0", inplace=True)


cited_patents_df = pd.DataFrame(data, columns=["patent_number", "cited_patent_number"])
data

,Unnamed: 0,patent_number,patent_date,patent_title,inventor_last_name,inventor_first_name,inventor_key_id,assignee_id,assignee_organization,assignee_lastknown_latitude,assignee_lastknown_longitude,assignee_latitude,assignee_location_id,assignee_longitude,assignee_key_id,cited_patent_number,citedby_patent_number,cpc_group_id
4,4,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,4904132,10452828.0,A41D
64,64,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,4991249,10452828.0,A41D
124,124,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,5386741,10452828.0,A41D
184,184,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,5497317,10452828.0,A41D
244,244,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,6244644,10452828.0,A41D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197448,197448,9992022,2018-06-05,Systems and methods for digital identity manag...,Czupek,Andrew,173180,25203305-3670-44b4-937d-eb687f8037b5,Northern Trust Corporation,41.8338,-87.6718,41.8338,41.8338|-87.6718,-87.6718,63537.0,8023648,10204339.0,G06F
197466,197466,9992022,2018-06-05,Systems and methods for digital identity manag...,Czupek,Andrew,173180,25203305-3670-44b4-937d-eb687f8037b5,Northern Trust Corporation,41.8338,-87.6718,41.8338,41.8338|-87.6718,-87.6718,63537.0,9049010,10204339.0,G06F
197862,197862,9998286,2018-06-12,Hardware blockchain consensus operating proced...,Greene,Kevin,1688488,79eeb049-c95b-4656-9f9e-01e4fe6e941d,Accenture Global Services Limited,53.3498,-6.26027,53.3498,53.3498|-6.26027,-6.26027,209358.0,5586313,10268817.0,G06F
197922,197922,9998286,2018-06-12,Hardware blockchain consensus operating proced...,Greene,Kevin,1688488,79eeb049-c95b-4656-9f9e-01e4fe6e941d,Accenture Global Services Limited,53.3498,-6.26027,53.3498,53.3498|-6.26027,-6.26027,209358.0,9679276,10268817.0,G06F


# Ähnlichkeitsberechnung
Im nächsten Schritt wird der Dataframe auf Basis der Patentnummern und der zitierten Patente pivotiert. Hierbei werden NULL-Werte mit 0 ersetzt.
Anschließend wird die Ähnlichkeit der Patentvektoren mit der Methode cosine_similarity berechnet und in einen neuen Dataframe geladen, 
welcher als Zeilen- und Spaltenindex die Patentnummern enthält.
Die entstehende Matrix wird als CSV-Datei exportiert

In [16]:
data["count"] = 1

display(data)

citation_matrix = data.pivot(index="patent_number", columns="cited_patent_number", values="count")
citation_matrix.fillna(0, inplace=True)

citation_similarity = cosine_similarity(citation_matrix)
sim_df = pd.DataFrame(citation_similarity, index=citation_matrix.index, columns=citation_matrix.index)

,Unnamed: 0,patent_number,patent_date,patent_title,inventor_last_name,inventor_first_name,inventor_key_id,assignee_id,assignee_organization,assignee_lastknown_latitude,assignee_lastknown_longitude,assignee_latitude,assignee_location_id,assignee_longitude,assignee_key_id,cited_patent_number,citedby_patent_number,cpc_group_id,count
4,4,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,4904132,10452828.0,A41D,1
64,64,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,4991249,10452828.0,A41D,1
124,124,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,5386741,10452828.0,A41D,1
184,184,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,5497317,10452828.0,A41D,1
244,244,10022613,2018-07-17,Smart device,Tran,Bao,442587,0,0,0,0,0,0,0,0,6244644,10452828.0,A41D,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197448,197448,9992022,2018-06-05,Systems and methods for digital identity manag...,Czupek,Andrew,173180,25203305-3670-44b4-937d-eb687f8037b5,Northern Trust Corporation,41.8338,-87.6718,41.8338,41.8338|-87.6718,-87.6718,63537.0,8023648,10204339.0,G06F,1
197466,197466,9992022,2018-06-05,Systems and methods for digital identity manag...,Czupek,Andrew,173180,25203305-3670-44b4-937d-eb687f8037b5,Northern Trust Corporation,41.8338,-87.6718,41.8338,41.8338|-87.6718,-87.6718,63537.0,9049010,10204339.0,G06F,1
197862,197862,9998286,2018-06-12,Hardware blockchain consensus operating proced...,Greene,Kevin,1688488,79eeb049-c95b-4656-9f9e-01e4fe6e941d,Accenture Global Services Limited,53.3498,-6.26027,53.3498,53.3498|-6.26027,-6.26027,209358.0,5586313,10268817.0,G06F,1
197922,197922,9998286,2018-06-12,Hardware blockchain consensus operating proced...,Greene,Kevin,1688488,79eeb049-c95b-4656-9f9e-01e4fe6e941d,Accenture Global Services Limited,53.3498,-6.26027,53.3498,53.3498|-6.26027,-6.26027,209358.0,9679276,10268817.0,G06F,1


Als nächstes wird ein threshold festegelegt, welcher die Ergebnisse auf einen Ähnlichkeitswert von über 0,6 beschränkt. Dies dient dazu, um lediglich Patente mit höherer Ähnlichkeit in die Analyse einfließen zu lassen.

In [5]:
citation_similarity = sim_df

cited_patents_df.drop_duplicates("patent_number")

cited_patent_list = set(cited_patents_df["patent_number"])
citation_similarity.index = citation_similarity.index.astype(str)

citation_similarity = citation_similarity[citation_similarity.index.isin(cited_patent_list)]
citation_similarity = citation_similarity[set(cited_patent_list)]
threshold = 0.6

citation_similarity_filtered = citation_similarity
citation_similarity_filtered.values[citation_similarity_filtered <= threshold] = 0

citsim_graph = nx.from_pandas_adjacency(citation_similarity_filtered,  create_using=nx.DiGraph)

# Vorbereitung der Visualisierung
Um den Knoten der Graphen weitere Informationen zu hinterlegen werden Dictionaries mit Informationen über das Anmelderunternehmen, die Patentnummer, die Zentralität und den Patenttitel erstellt. Diese werden dann den Knotenattributen zugeordnet. Außerdem wurde hierbei nur der in der Ausarbeitung dargestellte Graph mit den zehn Patenten berücksichtigt (siehe patents_to_analyze).

In [6]:
patents_to_analyze = ['10360191', '10447770', '10657526', '10505949', '10255108', '10608825', '10764325', '10382388', '10783128', '10790963']
patents_to_analyze_df = data[data['patent_number'].isin(patents_to_analyze)].drop_duplicates(subset='patent_number')

organization_dict = {}
for tuple in data.itertuples():    
    if tuple[2] in patents_to_analyze:
        if(tuple[9] == "0"):
            organization_dict[tuple[2]] = tuple[5]
        else:
            organization_dict[tuple[2]] = tuple[9]
nx.set_node_attributes(citsim_graph, organization_dict, "organization")

patent_id_dict = {tuple[2]:tuple[2] for tuple in data.itertuples()}
nx.set_node_attributes(citsim_graph, patent_id_dict, "patent_id")

degree = nx.degree_centrality(citsim_graph)
degree_dict = {key:value*1000 + 3 for key,value in degree.items()}
nx.set_node_attributes(citsim_graph, degree_dict, "centrality")


title_dict = {tuple[2]:tuple[4] for tuple in data.itertuples()}
nx.set_node_attributes(citsim_graph, title_dict, "patent_title")

# Ähnlichkeitsberechung der Patentitel
Zur weiteren Analyse wurde eine Ähnlichkeitsberechnung auf Grundlage der Patenttitel durchgeführt. Diese wurde mit der Open-Source-Bibliothek Spacy, wie im Anschluss zu sehen, ermittelt.

In [7]:
patent_title_sim_matrix = []
nlp = spacy.load("en_core_web_lg")
for patent1 in patents_to_analyze_df.itertuples():
    doc1 = nlp(patent1[4])    
    patent_title_sim_vector = []        
    for patent2 in patents_to_analyze_df.itertuples():        
        doc2 = nlp(patent2[4])                
        patent_similarity = doc1.similarity(doc2)
        patent_title_sim_vector.append(patent_similarity)  
        
    patent_title_sim_matrix.append(patent_title_sim_vector)
patent_title_sim_matrix_df = pd.DataFrame(patent_title_sim_matrix, index=patents_to_analyze_df['patent_number'], columns=patents_to_analyze_df['patent_number'])

Die zuvor berechneten Ähnlichkeiten wurden dann verwendet, um den Kanten des Graphen eine Färbung zu geben. Das entstehende Dictionary wird dann den Kantenattributen hinzugefügt.

In [8]:
edge_color_dict = {}
i = 0
col = 0

while i < len(patent_title_sim_matrix_df) - 1:
    j = 1 + col    
    while j < len(patent_title_sim_matrix_df):
        
        if (patent_title_sim_matrix_df.index[i], patent_title_sim_matrix_df.index[j]) in citsim_graph.edges:            
            if patent_title_sim_matrix_df.iloc[i,j] >= 0.8:
                edge_color_dict[(patent_title_sim_matrix_df.index[i], patent_title_sim_matrix_df.index[j])] = '#e0b004'
                edge_color_dict[(patent_title_sim_matrix_df.index[j], patent_title_sim_matrix_df.index[i])] = '#e0b004'
            elif patent_title_sim_matrix_df.iloc[i,j] >= 0.7:
                edge_color_dict[(patent_title_sim_matrix_df.index[i], patent_title_sim_matrix_df.index[j])] = '#d2e004'
                edge_color_dict[(patent_title_sim_matrix_df.index[j], patent_title_sim_matrix_df.index[i])] = '#d2e004'
            elif patent_title_sim_matrix_df.iloc[i,j] >= 0.6:
                edge_color_dict[(patent_title_sim_matrix_df.index[i], patent_title_sim_matrix_df.index[j])] = '#009687'
                edge_color_dict[(patent_title_sim_matrix_df.index[j], patent_title_sim_matrix_df.index[i])] = '#009687'
            elif patent_title_sim_matrix_df.iloc[i,j] >= 0.5:
                edge_color_dict[(patent_title_sim_matrix_df.index[i], patent_title_sim_matrix_df.index[j])] = '#09cf02'
                edge_color_dict[(patent_title_sim_matrix_df.index[j], patent_title_sim_matrix_df.index[i])] = '#09cf02'                           
        j += 1        
    col += 1
    i += 1
nx.set_edge_attributes(citsim_graph, edge_color_dict, 'edge_colors')

# Visualisierung
Abschließend wird der Graph mit dem Modul Networkx-Altair visualisiert. Eine interaktive Benutzung ist über das Menü mit den drei Punkten möglich. Dort können zudem weitere Daten, wie die zuvor genannten Knoteninformationen über die Tooltips abgerufen werden. Der analysierte Graph ist an den verschieden eingefärbten Kanten und Knoten zu erkennen.

In [9]:
alt.data_transformers.disable_max_rows()
chart = nxa.draw_networkx(
    citsim_graph,
    node_color='organization',
    #cmap= 'plasma',
    edge_color= 'edge_colors',
    node_size='centrality',
    node_tooltip = ["organization", "patent_id", "patent_title"],
    width = 3
).properties(
    width=800,
    height=800
).interactive()
chart



alt.LayerChart(...)

In der folgenden Zelle sind nun noch die einzelnen Titelähnlichkeiten zwischen allen Knoten zu sehen. Hierbei ist zu beachten, dass nicht alle Werte berücksichtigt wurden, sondern nur diejenigen, die auch eine Kante im Graphen bilden.

In [10]:
patent_title_sim_matrix_df

patent_number,10255108,10360191,10382388,10447770,10505949,10608825,10657526,10764325,10783128,10790963
patent_number,,,,,,,,,,
10255108,1.000000,0.662759,0.512098,0.617980,0.598023,0.554452,0.692299,0.713534,0.597446,0.728576
10360191,0.662759,1.000000,0.770007,0.634692,0.711426,0.643772,0.761132,0.782359,0.674112,0.709557
10382388,0.512098,0.770007,1.000000,0.441702,0.772037,0.786042,0.804407,0.712205,0.747571,0.549278
10447770,0.617980,0.634692,0.441702,1.000000,0.576281,0.502570,0.626100,0.617597,0.488098,0.649247
10505949,0.598023,0.711426,0.772037,0.576281,1.000000,0.691284,0.880854,0.799270,0.781997,0.764233
10608825,0.554452,0.643772,0.786042,0.502570,0.691284,1.000000,0.719788,0.668454,0.631549,0.508425
10657526,0.692299,0.761132,0.804407,0.626100,0.880854,0.719788,1.000000,0.812225,0.815907,0.694755
10764325,0.713534,0.782359,0.712205,0.617597,0.799270,0.668454,0.812225,1.000000,0.688028,0.770061
10783128,0.597446,0.674112,0.747571,0.488098,0.781997,0.631549,0.815907,0.688028,1.000000,0.633292
